In [ ]:
!git clone https://github.com/openai/shap-e

In [ ]:
%cd shap-e

In [ ]:
!pip install -e .

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [ ]:
batch_size = 1
guidance_scale = 15.0
prompt = "a red plane"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

In [ ]:
render_mode = 'nerf' 
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Создаем виджеты для ввода параметров
prompt_widget = widgets.Text(value='a red plane', description='Prompt:', disabled=False)
guidance_scale_widget = widgets.FloatSlider(value=15.0, min=1.0, max=30.0, step=0.1, description='Guidance Scale:', disabled=False)
render_mode_widget = widgets.Dropdown(options=['nerf', 'stf'], value='nerf', description='Render Mode:', disabled=False)
size_widget = widgets.IntSlider(value=64, min=32, max=256, step=1, description='Size:', disabled=False)

# Создаем кнопку для запуска генерации
generate_button = widgets.Button(description="Generate")

# Определяем функцию, которая будет вызываться при нажатии на кнопку
def on_generate_button_clicked(b):
    prompt = prompt_widget.value
    guidance_scale = guidance_scale_widget.value
    render_mode = render_mode_widget.value
    size = size_widget.value

    latents = sample_latents(
        batch_size=batch_size,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[prompt] * batch_size),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    cameras = create_pan_cameras(size, device)
    for i, latent in enumerate(latents):
        images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
        display(gif_widget(images))
